# Pandas

Материалы:
* Макрушин С.В. "Лекция 2: Библиотека Pandas"
* https://pandas.pydata.org/docs/user_guide/index.html#
* https://pandas.pydata.org/docs/reference/index.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Загрузите данные из файла `sp500hst.txt` и обозначьте столбцы в соответствии с содержимым: `"date", "ticker", "open", "high", "low", "close", "volume"`.

2. Рассчитайте среднее значение показателей для каждого из столбцов с номерами 3-6.

3. Добавьте столбец, содержащий только число месяца, к которому относится дата.

4. Рассчитайте суммарный объем торгов для одинаковых значений тикеров.

5. Загрузите данные из файла sp500hst.txt и обозначьте столбцы в соответствии с содержимым: "date", "ticker", "open", "high", "low", "close", "volume". Добавьте столбец с расшифровкой названия тикера, используя данные из файла `sp_data2.csv` . В случае нехватки данных об именах тикеров корректно обработать их.

## Лабораторная работа №2

### Базовые операции с `DataFrame`

In [1]:
import numpy as np
import pandas as pd

1.1 В файлах `recipes_sample.csv` и `reviews_sample.csv` находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами.

In [2]:
recipes = pd.read_csv("./data/data/recipes_sample.csv")
reviews = pd.read_csv("./data/data/reviews_sample.csv", parse_dates=["date"])
reviews

FileNotFoundError: [Errno 2] No such file or directory: './data/data/recipes_sample.csv'

1.2 Для каждой из таблиц выведите основные параметры:
* количество точек данных (строк);
* количество столбцов;
* тип данных каждого столбца.

In [ ]:
f"recipes: {recipes.shape} ", recipes.dtypes, f"reviews: {reviews.shape}", reviews.dtypes

1.3 Исследуйте, в каких столбцах таблиц содержатся пропуски. Посчитайте долю строк, содержащих пропуски, в отношении к общему количеству строк.

In [ ]:
recipes.isna().sum(), reviews.isna().sum()

In [ ]:
# (recipes.isna().sum().sum() + reviews.isna().sum().sum()) / (len(recipes.index) + len(reviews.index))

recipes_NA = recipes.isna().sum(axis=1)
reviews_NA = reviews.isna().sum(axis=1)
f"recipes: {round(recipes_NA[recipes_NA != 0].count() * 100 / len(recipes.index), 2)}%", \
f"reviews: {round(reviews_NA[reviews_NA != 0].count() * 100 / len(reviews.index), 7)}%"


1.4 Рассчитайте среднее значение для каждого из числовых столбцов (где это имеет смысл).

In [ ]:
# reviews.mean(), recipes.mean()
for i in ("n_ingredients", "minutes", "n_steps"):
    print(i, "-", recipes[i].mean())


f"Mean rating - {reviews['rating'].mean()}"

1.5 Создайте серию из 10 случайных названий рецептов.

In [ ]:
pd.Series(np.random.choice(recipes.loc[:, "name"], 10))

1.6 Измените индекс в таблице `reviews`, пронумеровав строки, начиная с нуля.

In [ ]:
reviews = reviews.set_index('Unnamed: 0').sort_index()

1.7 Выведите информацию о рецептах, время выполнения которых не больше 20 минут и кол-во ингредиентов в которых не больше 5.

In [ ]:
recipes[(recipes["minutes"] < 20) & (recipes["n_ingredients"] <= 5)]

### Работа с датами в `pandas`

2.1 Преобразуйте столбец `submitted` из таблицы `recipes` в формат времени. Модифицируйте решение задачи 1.1 так, чтобы считать столбец сразу в нужном формате.

In [ ]:
recipes["submitted"] = pd.DatetimeIndex(pd.to_datetime(recipes["submitted"]))
type(recipes["submitted"][0])

2.2 Выведите информацию о рецептах, добавленных в датасет не позже 2010 года.

In [ ]:
recipes[recipes["submitted"] < pd.to_datetime("2010-01-01")]

### Работа со строковыми данными в `pandas`

3.1 Добавьте в таблицу `recipes` столбец `description_length`, в котором хранится длина описания рецепта из столбца `description`.

In [ ]:
recipes["description_length"] = recipes["description"].str.len()
recipes

3.2 Измените название каждого рецепта в таблице `recipes` таким образом, чтобы каждое слово в названии начиналось с прописной буквы.

In [ ]:
recipes["name"].str.title()

3.3 Добавьте в таблицу `recipes` столбец `name_word_count`, в котором хранится количество слов из названии рецепта (считайте, что слова в названии разделяются только пробелами).

In [ ]:
recipes["name_word_count"] = recipes["name"].str.split().str.len()
recipes

### Группировки таблиц `pd.DataFrame`

4.1 Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

In [ ]:
recipes.groupby("contributor_id")["id"].count().sort_values()#.iloc[-1]

4.2 Посчитайте средний рейтинг к каждому из рецептов. Для скольких рецептов отсутствуют отзывы?

In [ ]:
reviews.groupby("recipe_id")["rating"].mean(), reviews.groupby("recipe_id")["rating"].mean().eq(0).sum()

4.3 Посчитайте количество рецептов с разбивкой по годам создания.

In [ ]:
recipes.groupby(pd.Grouper(key="submitted", freq="Y"))["id"].count()
# recipes.groupby(recipes["submitted"].dt.year)["name"].count()

### Объединение таблиц `pd.DataFrame`

5.1 При помощи объединения таблиц, создайте `DataFrame`, состоящий из четырех столбцов: `id`, `name`, `user_id`, `rating`. Рецепты без отзывов должны отсутствовать в данной таблице. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и выведя на экран строку из полученного `DataFrame`, содержащую информацию об этом отзыве.


In [ ]:
recipe_review = pd.merge(recipes, reviews, left_on="id", right_on="recipe_id")
recipe_review = recipe_review.dropna(how="any")
recipe_review.drop(recipe_review.columns.difference(["id", "name", "user_id", "rating"]), 1, inplace=True)
recipe_review = recipe_review[["id", "name", "user_id", "rating"]]
# recipe_review = recipe_review.drop(recipe_review[recipe_review["rating"] == 0].index)
# recipe_review[recipe_review["rating"] == 0]
recipe_review

5.2 При помощи объединения таблиц и группировок, создайте `DataFrame`, состоящий из трех столбцов: `recipe_id`, `name`, `review_count`. У рецептов, для которых отсутствуют отзывы, в соответствущем столбце должен быть указан 0. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и выведя на экран строку из полученного `DataFrame`, содержащую информацию об этом отзыве.

In [ ]:
recipe_name_review = pd.merge(recipes, reviews, left_on="id", right_on="recipe_id")

review_count = recipe_name_review.groupby("recipe_id")["rating"].count()
review_count.columns = ["recipe_id", "review_count"]

review_count = pd.merge(recipes, review_count, how="inner", left_on="id", right_on="recipe_id")
review_count.drop(review_count.columns.difference(["id", "name", "rating"]), 1, inplace=True)

review_count.columns = ["name", "recipe_id", "review_count"]
(review_count["review_count"] == 0).sum()


5.3. Выясните, отзывы, добавленные в каком году, имеют наименьший средний рейтинг?

In [ ]:
reviews_and_score = reviews.groupby(reviews['date'].dt.year)['rating'].mean()
f"Min year: {reviews_and_score.idxmin()} | min score: {reviews_and_score.min()}"

### Сохранение таблиц `pd.DataFrame`

6.1 Отсортируйте таблицу в порядке убывания величины столбца `name_word_count` и сохраните результаты выполнения заданий 3.1-3.3 в csv файл. 

In [ ]:
recipes = recipes.sort_values("name_word_count")
recipes.to_csv("recipes_out.csv", sep=";", encoding="UTF-8")

6.2 Воспользовавшись `pd.ExcelWriter`, cохраните результаты 5.1 и 5.2 в файл: на лист с названием `Рецепты с оценками` сохраните результаты выполнения 5.1; на лист с названием `Количество отзывов по рецептам` сохраните результаты выполнения 5.2.

In [ ]:
with pd.ExcelWriter("recipe_review.xlsx") as writer:
    recipe_review.to_excel(writer, sheet_name="Рецепты с оценками")
    review_count.to_excel(writer, sheet_name="Количество отзывов по рецептам")